In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mental-health-dataset/Mental Health Dataset.csv


In [2]:
df=pd.read_csv("/kaggle/input/mental-health-dataset/Mental Health Dataset.csv")
df

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,8/27/2014 11:29,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,8/27/2014 11:31,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,8/27/2014 11:32,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,8/27/2014 11:43,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,7/27/2015 23:25,Male,United States,Business,Yes,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,Maybe,Not sure
292360,8/17/2015 9:38,Male,South Africa,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes
292361,8/25/2015 19:59,Male,United States,Business,No,Yes,No,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,No
292362,9/26/2015 1:07,Male,United States,Business,No,Yes,Yes,15-30 days,No,Maybe,No,Low,Yes,No,Maybe,No,Yes


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC, SVC
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier

from category_encoders import LeaveOneOutEncoder
from category_encoders.target_encoder import TargetEncoder
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc

import warnings
warnings.filterwarnings("ignore")

In [4]:
le=LabelEncoder()

le=LabelEncoder()

column_object=df.select_dtypes("object").columns
column_object

for col in column_object:
    df[col]=le.fit_transform(df[col])

df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292364 entries, 0 to 292363
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype
---  ------                   --------------   -----
 0   Timestamp                292364 non-null  int64
 1   Gender                   292364 non-null  int64
 2   Country                  292364 non-null  int64
 3   Occupation               292364 non-null  int64
 4   self_employed            292364 non-null  int64
 5   family_history           292364 non-null  int64
 6   treatment                292364 non-null  int64
 7   Days_Indoors             292364 non-null  int64
 8   Growing_Stress           292364 non-null  int64
 9   Changes_Habits           292364 non-null  int64
 10  Mental_Health_History    292364 non-null  int64
 11  Mood_Swings              292364 non-null  int64
 12  Coping_Struggles         292364 non-null  int64
 13  Work_Interest            292364 non-null  int64
 14  Social_Weakness          292364 non-

Timestamp                  0
Gender                     0
Country                    0
Occupation                 0
self_employed              0
family_history             0
treatment                  0
Days_Indoors               0
Growing_Stress             0
Changes_Habits             0
Mental_Health_History      0
Mood_Swings                0
Coping_Struggles           0
Work_Interest              0
Social_Weakness            0
mental_health_interview    0
care_options               0
dtype: int64

In [5]:
df = pd.get_dummies(df, columns=["Country"], drop_first=True)
df

,Timestamp,Gender,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,...,Country_25,Country_26,Country_27,Country_28,Country_29,Country_30,Country_31,Country_32,Country_33,Country_34
0,39,0,1,2,0,1,0,2,1,2,...,False,False,False,False,False,False,False,False,False,True
1,41,0,1,2,1,1,0,2,1,2,...,False,False,False,False,False,False,False,False,False,True
2,42,0,1,2,1,1,0,2,1,2,...,False,False,False,False,False,False,False,False,False,True
3,47,0,1,0,1,1,0,2,1,2,...,False,False,False,False,False,False,False,False,False,True
4,52,0,1,0,1,1,0,2,1,2,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292359,36,1,0,1,1,1,1,1,0,1,...,False,False,False,False,False,False,False,False,False,True
292360,37,1,0,0,1,1,1,1,0,1,...,False,False,False,False,True,False,False,False,False,False
292361,38,1,0,0,1,0,1,1,0,1,...,False,False,False,False,False,False,False,False,False,True
292362,570,1,0,0,1,1,1,1,0,1,...,False,False,False,False,False,False,False,False,False,True


In [6]:
df.dropna(inplace=True)

X=df.drop(["treatment","Timestamp"],axis=1)
y=df["treatment"]

X,y

X.shape
X

X = X.astype(int)
# X_test = X_test.astype(int)
X
y

0         1
1         1
2         1
3         1
4         1
         ..
292359    1
292360    1
292361    0
292362    1
292363    1
Name: treatment, Length: 292364, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
import torch
from torch import nn

In [9]:
# X_train.dtypes
type(X_train)


pandas.core.frame.DataFrame

In [10]:

X_train

,Gender,Occupation,self_employed,family_history,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,...,Country_25,Country_26,Country_27,Country_28,Country_29,Country_30,Country_31,Country_32,Country_33,Country_34
8229,0,1,2,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
53959,1,2,0,0,4,1,2,2,2,0,...,0,0,0,0,0,0,0,0,0,1
124067,1,0,0,1,3,1,2,1,2,1,...,0,0,0,0,0,0,0,0,1,0
150820,1,4,0,0,1,1,2,0,0,0,...,0,0,0,0,0,0,0,0,0,1
120868,1,4,0,1,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1,0,0,0,2,0,1,0,2,0,...,0,0,0,0,0,0,0,0,0,1
259178,1,2,0,0,1,2,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0
131932,1,4,0,0,3,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
146867,1,0,0,1,3,0,2,2,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
type(y_train)
type(X_test)


pandas.core.frame.DataFrame

In [12]:
import torch

X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).squeeze()
y_test = torch.tensor(y_test.values, dtype=torch.float32).squeeze()



In [13]:
X_train.shape
type(X_train)

torch.Tensor

In [14]:
class Mental_V0(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1=nn.Linear(in_features=48,out_features=300)
        self.relu = nn.ReLU()
        self.layer2=nn.Linear(in_features=300,out_features=200)
        self.relu = nn.ReLU()
        self.layer3=nn.Linear(in_features=200,out_features=100)
        self.layer4=nn.Linear(in_features=100,out_features=20)
        self.layer5=nn.Linear(in_features=20,out_features=1)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x= self.relu(self.layer4(x))
        x= self.layer5(x)
          
        return x


In [15]:
model_0=Mental_V0()
# model_0.state_dict()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)
y_train = y_train.to(DEVICE)
y_test = y_test.to(DEVICE)

type(X_train)
model_0=Mental_V0().to(DEVICE)


# model_0.state_dict()



In [16]:
loss_fn=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(params=model_0.parameters(),lr=0.01)


In [17]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [18]:
epochs=500

for epoch in range(epochs):
    model_0.train()
    y_logits=model_0(X_train).squeeze()
    y_pred=torch.round(torch.sigmoid(y_logits))
    loss_train=loss_fn(y_logits,y_train)
    acc_train=accuracy_fn(y_pred,y_train)


    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    model_0.eval()
    with torch.inference_mode():
        y_logits_test=model_0(X_test).squeeze()
        loss_test=loss_fn(y_logits_test,y_test)
        y_pred_test=torch.round(torch.sigmoid(y_logits_test))
        acc_test=accuracy_fn(y_pred_test,y_test)
    if (epoch+1) % 50 == 0:
        print(f"Epoch {epoch+1}/{epochs}|"
              f"Train loss:{loss_train:.4f}| Train acc: {acc_train:.2f}% | "
              f"Test loss:{loss_test:.4f}| Test acc: {acc_test:.2f}%")
        
    

Epoch 50/500|Train loss:0.5130| Train acc: 73.90% | Test loss:0.5158| Test acc: 73.69%
Epoch 100/500|Train loss:0.4490| Train acc: 77.53% | Test loss:0.4470| Test acc: 77.60%
Epoch 150/500|Train loss:0.4174| Train acc: 78.68% | Test loss:0.4220| Test acc: 78.44%
Epoch 200/500|Train loss:0.4129| Train acc: 78.78% | Test loss:0.4179| Test acc: 78.36%
Epoch 250/500|Train loss:0.4116| Train acc: 78.85% | Test loss:0.4242| Test acc: 77.79%
Epoch 300/500|Train loss:0.4100| Train acc: 78.90% | Test loss:0.4222| Test acc: 78.05%
Epoch 350/500|Train loss:0.4096| Train acc: 78.91% | Test loss:0.4263| Test acc: 77.89%
Epoch 400/500|Train loss:0.4101| Train acc: 78.90% | Test loss:0.4272| Test acc: 77.76%
Epoch 450/500|Train loss:0.4092| Train acc: 78.94% | Test loss:0.4306| Test acc: 77.72%
Epoch 500/500|Train loss:0.4087| Train acc: 78.97% | Test loss:0.4322| Test acc: 77.72%


In [19]:
y_train.unique()

tensor([0., 1.], device='cuda:0')

In [20]:
import torch

print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU Available: True
GPU Name: Tesla P100-PCIE-16GB


In [21]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)
y_train = y_train.to(DEVICE)
y_test = y_test.to(DEVICE)

type(X_train)
model_0=Mental_V0().to(DEVICE)


# model_0.state_dict()
